In [1]:
#Dependecies
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as np
import seaborn as sns
import requests
from os import getcwd


## Factores for Country Analysis

In [2]:
# calculate CAGR for population, cels, and gdp per capita
def CAGR(first, last, periods):
    return round(((last/first)**(1/periods)-1)*100,2)

In [3]:
import os 
os.getcwd()

'C:\\Users\\A\\Desktop\\final-ml-project-covid19'

In [4]:
# !dir C:\Users\A
PATH_TO_FILES='C:\\Users\\A\\Desktop\\final-ml-project-covid19\\'

In [5]:
gdp_pc=pd.read_csv(PATH_TO_FILES+'gdp_percapita.csv')
gdp_pc=gdp_pc.dropna(how='all')
gdp_pc=gdp_pc.dropna(subset = ['2016'])
# gdp_pc=gdp_pc.fillna(axis=1,method='ffill')
gdp_pc=gdp_pc.fillna(axis=1,method='ffill') # using last valid year data before dropping rows
gdp_pc['CAGR_GDP']=gdp_pc[['2015','2019']].apply(lambda x: CAGR(x['2015'],x['2019'],4),axis=1)
# change the df
gdp_pc=gdp_pc.drop(columns=['2015','2016','2017','2018'])
gdp_pc['2019']=pd.to_numeric(gdp_pc['2019'],downcast='float')
gdp_pc['2019']=gdp_pc['2019'].apply(lambda x: round(x,2))
gdp_pc.rename(columns={'2019': 'GDPPC'}, inplace=True)
gdp_pc.head() # holds GDP per capita

FileNotFoundError: [Errno 2] File C:\Users\A\Desktop\final-ml-project-covid19\gdp_percapita.csv does not exist: 'C:\\Users\\A\\Desktop\\final-ml-project-covid19\\gdp_percapita.csv'

# World Bank Data
- Celular Usde

In [ ]:
cel_p100=pd.read_csv(PATH_TO_FILES+'cels_percapita.csv')
cel_p100=cel_p100.dropna(how='all')
cel_p100=cel_p100.dropna(subset = ['2016'])
cel_p100=cel_p100.fillna(axis=1,method='ffill')
cel_p100['CAGR_CEL']=cel_p100[['2015','2019']].apply(lambda x: CAGR(x['2015'],x['2019'],4),axis=1)
cel_p100=cel_p100.drop(columns=['2015','2016','2017','2018'])
cel_p100['2019']=pd.to_numeric(cel_p100['2019'],downcast='float')
cel_p100['2019']=cel_p100['2019'].apply(lambda x: round(x,2))
cel_p100.rename(columns={'2019': 'CELSP100'}, inplace=True)
cel_p100.head()

## Country Population

In [ ]:
# value in thousands
country_pop=pd.read_csv(PATH_TO_FILES+'wbcountry_population.csv')
country_pop=country_pop.dropna(how='all')
country_pop=country_pop.dropna(subset = ['2016'])
country_pop=country_pop.fillna(axis=1,method='ffill')
country_pop['CAGR_POP']=country_pop[['2015','2019']].apply(lambda x: CAGR(x['2015'],x['2019'],4),axis=1)
country_pop=country_pop.drop(columns=['2015','2016','2017','2018'])
country_pop.rename(columns={'2019': 'POPU'}, inplace=True)
country_pop.head()

### Combine Features

In [ ]:
country_data=gdp_pc.merge(cel_p100,how='inner',on='Country')
country_data=country_data.merge(country_pop,how='inner',on='Country')
country_data.head()

In [ ]:
country_data=country_data[['Country','Country_Code','GDPPC',
                          'CAGR_GDP','CELSP100','CAGR_CEL','POPU','CAGR_POP']]
country_data.head()

In [ ]:
country_area=pd.read_csv(PATH_TO_FILES+'countries_area.csv')
country_area=country_area[['name','cca2','cioc','region','languages','latlng','landlocked','area']]
country_area['langs']=country_area['languages'].apply(lambda x:len(str(x).split(",")))
country_area['lat']=country_area['latlng'].apply(lambda x:float(str(x).split(",")[0]))
country_area['lng']=country_area['latlng'].apply(lambda x:float(str(x).split(",")[1]))
country_area['landlocked'].fillna(0.0,inplace=True)
country_area['landlocked']=country_area['landlocked'].astype('int')
# country_area[country_area['cioc'].isna()] # returns little islands, so removing them
country_area=country_area.dropna(how='any')
country_area.reset_index(inplace=True,drop=True) # reseting index
country_area.head()

In [ ]:
country_data=country_data.merge(country_area,how='inner',left_on='Country_Code',right_on='cioc')
country_data['POPU']=country_data['POPU']/1000000  # pop per million
country_data.head() # area in square miles

In [ ]:
google_mr=pd.read_csv(PATH_TO_FILES+'Global_Mobility_Report.csv')

In [ ]:
# data on mobiliy by Google and has by country too
'''
by region and different categories
'''

google_mr=google_mr[['country_region_code','country_region','sub_region_1',
                     'sub_region_2','date',
                     'retail_and_recreation_percent_change_from_baseline']]
google_mr=google_mr.rename(columns={'retail_and_recreation_percent_change_from_baseline':'G_MDR'})
google_mr['date']=pd.to_datetime(google_mr['date'])
google_mr.reset_index(inplace=True,drop=True) # reseting index
google_mr=google_mr[google_mr['sub_region_1'].isna()] # only display country data not states

In [ ]:
'''
if you wish to plot date per country you would need to group as dates repeat per country
'''
google_mr[google_mr['country_region']=='United States']['G_MDR'].plot()

In [ ]:
GMR_df=google_mr.groupby('country_region')['G_MDR'].agg(['min','max'])

In [ ]:
GMR_df['Country']=GMR_df.index

In [ ]:
GMR_df.rename(columns={'min':'G_MDR_MIN','max':'G_MDR_MAX'},inplace=True)

In [ ]:
country_data=country_data.merge(GMR_df,how='left',on='Country')
# list of countries with no google data tracking that will remove
countries_nogdata=country_data[country_data.G_MDR_MIN.isna()].values
country_data=country_data[~country_data.G_MDR_MIN.isna()]

In [ ]:
 # need to create index to be able to save as dictionary much easier
country_data=country_data.set_index('Country',drop=False)

c_1M=country_data['POPU'].to_dict() # country pop per million
#dropping Canada and Australia as stats on region basis makes it hard
del c_1M['Canada']
del c_1M['Australia']
c_1M['US']=c_1M.pop('United States')
c_1M

In [ ]:
df_ts_COVID_d=pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
# del df_ts_COVID['Province/State']
del df_ts_COVID_d['Lat']
del df_ts_COVID_d['Long']
df_ts_COVID_d=df_ts_COVID_d.T
columns_to_replace={}
c_province=df_ts_COVID_d[0:1].values[0]
c_countries=df_ts_COVID_d[1:2].values[0]
# df_ts_COVID_d.head()
for c in df_ts_COVID_d[0:1].columns.values:
    if str(c_province[int(c)])!='nan':
           columns_to_replace[c]=str(c_countries[int(c)])+'-' + str(c_province[int(c)])
    else:
           columns_to_replace[c]=str(c_countries[int(c)])
# columns_to_replace
df_ts_COVID_d=df_ts_COVID_d.rename(columns=columns_to_replace)
df_ts_COVID_d=df_ts_COVID_d.drop(['Province/State', 'Country/Region'])
df_ts_COVID_d.index=pd.to_datetime(df_ts_COVID_d.index)

In [ ]:
for c in c_1M.keys():
    df_ts_COVID_d[c]=df_ts_COVID_d[c]/c_1M[c] # divide by population to ratio it to make it comparable to other countries
df_ts_COVID_d=df_ts_COVID_d[c_1M.keys()]

In [ ]:
df_ts_COVID_d=df_ts_COVID_d.rename(columns={'US':'United States'})
# gets the last day of infections for all the countries
df_ts_COVID_d[-1:]

In [ ]:
df_ts_COVID_d.tail(10).mean()

In [ ]:
df_ts_1MD=df_ts_COVID_d[-1:]
df_ts_1MD=df_ts_1MD.reset_index(drop=True)
df_ts_1MD=df_ts_1MD.T
df_ts_1MD=df_ts_1MD.rename(columns={0:'DPM'})
df_ts_1MD['Country']=df_ts_1MD.index
df_ts_1MD=df_ts_1MD.reset_index(drop=True)

In [ ]:
country_data=country_data.reset_index(drop=True)

In [ ]:
last_country_data=country_data.merge(df_ts_1MD,how='left',left_on='Country',right_on='Country')

In [ ]:
last_country_data[last_country_data['DPM'].isna()]# aus and cad dropped as these do not have data

In [ ]:
last_country_data.dropna(inplace=True)

In [ ]:
last_country_data

In [ ]:
'''
final feature selection
1) columns: Country, 
country
'''

last_country_data=last_country_data[['Country', 'Country_Code', 'GDPPC', 'CAGR_GDP', 'CELSP100', 'CAGR_CEL',
       'POPU', 'CAGR_POP', 'landlocked', 'area', 'langs', 'lat', 'lng', 'G_MDR_MIN',
       'G_MDR_MAX','DPM']]

In [ ]:
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [ ]:
def status_label(x):
    if x <50:
        return int(1)
    elif x<100:
        return int(2)
    elif x <200:
        return int(3)
    else:
        return int(4)

In [ ]:
last_country_data['Status_Label']=last_country_data['DPM'].apply(lambda x: status_label(x))

In [ ]:
last_country_data

In [ ]:
last_country_data['Status_Label'].hist()
plt.title('Histogram of Status Label')
plt.ylabel('# of instances of Label')
plt.ylabel('Label')
plt.show()

In [ ]:
features_country=['GDPPC', 'CAGR_GDP', 'CELSP100', 'CAGR_CEL',
       'POPU', 'CAGR_POP', 'landlocked', 'area', 'langs',
       'G_MDR_MIN', 'G_MDR_MAX', 'Status_Label']

In [ ]:
X=last_country_data[features_country[:-1]].values
y=last_country_data[features_country[-1]].values

In [ ]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.5,random_state=0)

In [ ]:
X_km=last_country_data[features_country[:-1]].values
y_km=last_country_data['Status_Label'].values
# kmeans = KMeans(n_clusters=4, random_state=0).fit(X_train)
kmeans = KMeans(n_clusters=4, random_state=0)
kmeans.fit(X_km)
# y_pred = kmeans.fit_predict(X_test)

In [ ]:
kmeans.labels_

In [ ]:
fig,ax = plt.subplots(figsize=(4, 3))
labels = kmeans.labels_

ax.scatter(X_km[:, 4], X_km[:, 2])
# ax.scatter([0,2,3], [2,3,1])
ax.set_xlabel('GDP')
ax.set_ylabel('Celsp per 1K')
ax.set_title('K-means Country COVID-19')
# ax.dist = 12

In [ ]:
df_eval=pd.DataFrame({'y_label':y_km,'y_pred':kmeans.labels_ + 1}) # adding 1 to make same
df_eval['Success']=(df_eval['y_label']==df_eval['y_pred'])

In [ ]:
df_eval

In [ ]:
print(f'Number of records in training: {len(kmeans.labels_)}')

In [ ]:
print(metrics.classification_report(y_km,kmeans.labels_))

In [ ]:
y_pred=kmeans.fit_predict(X_test)

In [ ]:
df_eval=pd.DataFrame({'y_label':y_test,'y_pred':kmeans.labels_ + 1}) # adding 1 to make same
df_eval['Success']=(df_eval['y_label']==df_eval['y_pred'])

In [ ]:
df_eval

In [ ]:
df_eval.Success.value_counts() # k-means has a hard time predicting anything, and allocating to right group

In [ ]:
from sklearn.svm import SVC

In [ ]:
reg_svc = SVC()
reg_svc.fit(X_train, y_train)
y_pred = reg_svc.predict(X_test)

In [ ]:
df_eval=pd.DataFrame({'y_label':y_test,'y_pred':y_pred})
df_eval['Correct']=(df_eval['y_label']==df_eval['y_pred'])
print(df_eval['Correct'].value_counts())
print("-"*100,'\nClassification Report\n')
print(metrics.classification_report(y_test, y_pred))
print('Accuracy Score is True / (True + False)\n Classes 2 and 3 are not being predicted right')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0,1))

In [ ]:
scaler.fit(X_train)

In [ ]:
X_train_scaled=scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
reg_svc = SVC()
reg_svc.fit(X_train_scaled, y_train)
y_pred = reg_svc.predict(X_test_scaled)

In [ ]:
df_eval=pd.DataFrame({'y_label':y_test,'y_pred':y_pred})
df_eval['Correct']=(df_eval['y_label']==df_eval['y_pred'])

print(df_eval['Correct'].value_counts())
print("-"*100,'\nClassification Report using MinMax Scaler\n')
print(metrics.classification_report(y_test, y_pred))
print('Accuracy Score is True / (True + False)\n Classes 2 and 3 are not being predicted right\nMinMax not that much more accurate')

# Visuals

In [ ]:
# fix,ax=plt.subplots(figsize=(20,20))
g=sns.pairplot(data=last_country_data[last_country_data['Status_Label']==4]
             ,x_vars=['GDPPC'],y_vars=['DPM'],hue='Country')
g.fig.set_size_inches(20,12)

In [ ]:
last_country_data['DPM_INT']=last_country_data['DPM'].astype('int')

In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter(data_frame=last_country_data[last_country_data['Status_Label']==4], x="GDPPC", y="DPM", text="Country", size_max=60)

fig.update_traces(textposition='top center')

fig.update_layout(
    height=800,
    title_text='modify title'
)

fig.show()


In [ ]:
'''
This article explains the phenomenon seen above:
https://www.politico.eu/article/in-defense-of-belgium-coronavirus-covid19-pandemic-response/
'''

In [ ]:
fig = px.scatter(data_frame=last_country_data[last_country_data['Status_Label'].isin([3,4])], 
                 x="CAGR_GDP", y="CAGR_CEL", size="DPM_INT", text="Country",color="Status_Label",
                template='ggplot2')

fig.update_traces(textposition='top center')

fig.update_layout(
    height=800,
    title_text='Cellphone Growth to GDP Growth\nCOVID Status on Bubble Scale'
#     plot_bgcolor='rgba(248,249,249)'
    
)
fig.show()

In [ ]:
fig = px.scatter(data_frame=last_country_data[last_country_data['Status_Label'].isin([3,4])], 
                 x="GDPPC", y="DPM", size="DPM_INT", text="Country",color="Status_Label",
                template='ggplot2')

fig.update_traces(textposition='top center')

fig.update_layout(
    height=800,
    title_text='Death per Million by GDP per Capita'
#     plot_bgcolor='rgba(248,249,249)'
    
)
fig.show()

In [ ]:
fig = px.scatter(data_frame=last_country_data, x="GDPPC", y="DPM", size="DPM_INT", text="Country",
                template='ggplot2')

fig.update_traces(textposition='top center',textfont_size=10)

fig.update_layout(
    height=800,
    title_text='Death per Million to GDP Per Capita'
)

fig.show()

In [ ]:
fig = px.scatter_geo(data_frame=last_country_data[last_country_data['Status_Label'].isin([2,3,4])],
                     hover_name="Country", size="DPM_INT",lat='lat',lon='lng',color='Status_Label')


fig.update_layout(
    height=800,
#     margin={"r":0,"t":15,"l":0,"b":0},
    mapbox_style="open-street-map",
    title_text='Death per Million Category 2-4'
)

fig.show()

In [ ]:
fig,ax=plt.subplots(figsize=(12,12))
ax.scatter(data=last_country_data[last_country_data['Status_Label']==2],
                                x='CAGR_POP', y='CAGR_CEL',c='b', alpha=0.75,s='DPM_INT',label='Categ-2')

ax.scatter(data=last_country_data[last_country_data['Status_Label']==3],
                                x='CAGR_POP', y='CAGR_CEL',c='g', alpha=0.5,s='DPM_INT',label='Categ-3',
          marker='s')

ax.scatter(data=last_country_data[last_country_data['Status_Label']==4],
                                x='CAGR_POP', y='CAGR_CEL',c='r', alpha=0.35,s='DPM_INT',label='Categ-4',
          marker='^')
# last_country_data[last_country_data['Status_Label'].isin([3,4])]
ax.set_xlabel('Population Growth')
ax.set_ylabel('Celular Phones Growth')
ax.legend()
plt.show()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
reg_log = LogisticRegression(max_iter=5000,multi_class='ovr')
reg_log.fit(X_train, y_train)
y_pred = reg_log.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test, y_pred))

In [ ]:
# Linear Regression
from sklearn.linear_model import LinearRegression

In [ ]:
linear_reg=LinearRegression(fit_intercept=False)
linear_reg.fit(X_train, y_train)
y_pred = reg_log.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test, y_pred))

In [ ]:
print(r2_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import roc_auc_score, r2_score

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
reg_knn = KNeighborsClassifier()
reg_knn.fit(X_train, y_train)
y_pred = reg_knn.predict(X_test)

In [ ]:
print(metrics.classification_report(y_test, y_pred))

In [ ]:
MIN_DPM=5 # MIN DPM
df_Country={} # hold several dataframes

In [ ]:
df_DBC=df_ts_COVID_d[last_country_data.Country.values].copy()
df_DBC['United States'][df_DBC['United States']>MIN_DPM].plot()

In [ ]:
df_DBC['Brazil'][df_DBC['Brazil']>MIN_DPM].plot()
df_DBC['Colombia'][df_DBC['Colombia']>MIN_DPM].plot(linewidth=4,color='r')
df_DBC['Argentina'][df_DBC['Argentina']>MIN_DPM].plot()
df_DBC['Ecuador'][df_DBC['Ecuador']>MIN_DPM].plot(marker='^')
plt.legend()

In [ ]:
c_1M['United States']=c_1M['US']

In [ ]:
del c_1M['US']
c_1M


In [ ]:
for k in c_1M.keys():
    days=df_DBC[k][df_DBC[k]>MIN_DPM].count()
    if days !=0:
#         print(k)
        # skip any country with no data, so only ones with days > 0
        df_c=pd.DataFrame({
            'TP':[i for i in range(1,days+1)],
            k:df_DBC[k][df_DBC[k]>MIN_DPM].values   # if you wish to keep dates, then remove .values
        }
        )
        df_Country[k]=df_c

In [ ]:
df_DBC_TP=pd.DataFrame({'PLACEHOLDER':range(0,200)},index=range(0,200)) # hold all TP to measure them all with correlations

In [ ]:
for k in df_Country.keys():
    df_DBC_TP=df_DBC_TP.merge(df_Country[k],how='left',left_index=True,right_index=True)
    df_DBC_TP[k]=pd.to_numeric(df_DBC_TP[k],downcast='float')
del df_DBC_TP['TP_y']
del df_DBC_TP['TP_x']
del df_DBC_TP['PLACEHOLDER']
df_DBC_TP=df_DBC_TP.dropna(how='all')
# df_DBC_TP=df_DBC_TP.dropna()

In [ ]:
df_Country['Brazil']['Brazil'].plot()
df_Country['United States']['United States'].plot()

In [ ]:
n_countries=last_country_data[['Country','Country_Code']][(last_country_data['Status_Label']==4) & (last_country_data['DPM']> MIN_DPM)].count()[0]
print(n_countries)

In [ ]:
fig,axs=plt.subplots(nrows=4,ncols=1,figsize=(20,40),sharex=False) # 4 categories, so 4 charts side by side
# print(axs.shape)
fig.suptitle(f'Countries with COVID DPM Over {MIN_DPM}\nData {len(df_Country.keys())}')
for i in range(1,5):
    # start with categories
    n_countries=last_country_data[['Country','Country_Code']][(last_country_data['Status_Label']==i) & (last_country_data['DPM']> MIN_DPM)].count()[0]
#     print(f'j={str(i)} and # of countries: {str(n_countries)}')
    for j in range(1,n_countries):
        country=last_country_data[['Country','Country_Code']][(last_country_data['Status_Label']==i) & (last_country_data['DPM']> MIN_DPM)].values[j][0]
        axs[i-1].plot(df_Country[country][country].index,df_Country[country][country],
                    label=last_country_data[['Country','Country_Code']][(last_country_data['Status_Label']==i) & (last_country_data['DPM']> MIN_DPM)].values[j][1])
        axs[i-1].legend()
        axs[i-1].set_xlabel('Time Period - Days since first 5 DPM')
        axs[i-1].set_ylabel('Death Per Million')
        axs[i-1].grid(b=True,alpha=0.25)

In [ ]:
fig,axs=plt.subplots()
fig.suptitle('All Countries with DPM > 5')

for k in df_Country.keys():
#     print(k)
    df_Country[k][k].plot(label=k)
axs.legend(bbox_to_anchor=(1.2, 1.15))
axs.set_xlabel('Time Period - Days since first 5 DPM')
axs.set_ylabel('Death Per Million')
plt.show()

## Correlations

In [ ]:
pd.plotting.scatter_matrix(df_DBC_TP, figsize=(20, 20),)
plt.show()

In [ ]:
fig,axs=plt.subplots(nrows=4,ncols=1,figsize=(20,40),sharex=False) # 4 categories, so 4 charts side by side
# print(axs.shape)
fig.suptitle(f'Countries(correlations) with COVID DPM Over {MIN_DPM}\nData {len(df_Country.keys())}')
for i in range(1,5):
    # start with categories
    n_countries=last_country_data[['Country','Country_Code']][(last_country_data['Status_Label']==i) & (last_country_data['DPM']> MIN_DPM)].count()[0]
    countries_ingroup=last_country_data[['Country','Country_Code']][(last_country_data['Status_Label']==i) & (last_country_data['DPM']> MIN_DPM)].iloc[:,0].values
    countries_ingroup_labels=last_country_data[['Country','Country_Code']][(last_country_data['Status_Label']==i) & (last_country_data['DPM']> MIN_DPM)].iloc[:,1].values
    
    df_DBC_TP_corr=df_DBC_TP[countries_ingroup].corr()
    
    np_country_corr=df_DBC_TP_corr.values
    
    mask = np.zeros_like(np_country_corr)
    
    

    mask[np.triu_indices_from(mask)] = True
    
#     pos = axs[i-1].imshow(df_DBC_TP_corr,cmap='spring',mask=mask,vmax=1,square=False)
    sns.heatmap(np_country_corr, mask=mask, vmax=1, square=True,
                cmap='spring',ax=axs[i-1],annot=True,linecolor='#435267'
               ,annot_kws={"size": 8})
    
    axs[i-1].set_xticks(np.arange(len(countries_ingroup_labels)))
    axs[i-1].set_xticklabels(countries_ingroup_labels)
    axs[i-1].set_yticks(np.arange(len(countries_ingroup_labels)))
    axs[i-1].set_yticklabels(countries_ingroup_labels)
#     axs[i-1].colorbar()
    axs[i-1].set_title(f'Group: {i}')